# Computational Simulation of Equivalence Class Formation Using the go/no-go Procedure with Compound Stimuli

### Abstract
Research about equivalence has commonly utilized human participants as experimental subjects. More recently, computational models have been capable of reproducing performances observed in experiments with humans. The computational model often utilized is called RELNET, and it simulates training and testing trials of conditional relations using the matching-to-sample procedure (MTS). The differentiation between sample stimulus and comparison stimuli, indispensable in MTS, implies operational difficulties for simulations. For this reason, new studies seek to utilize alternative procedures to MTS, which do not differentiate the functions of the antecedent stimuli. This work evaluated the possibility of developing a new computational model to simulate equivalence class formation using the go/no-go procedure with compound stimuli. In Experiment 1, artificial neural networks were utilized to simulate training of the AB and BC relations as well as the testing of the AC relation. The results showed that four out of six runs demonstrated equivalence class formation. Experiment 2 evaluated whether the additional class training performed in Experiment 1, which was analogous to the simulation of pre-experimental experience of human participants, would be essential for simulating the establishment of equivalence classes. It was found that it was not possible to simulate equivalence class formation without the additional class training. Altogether, the experiments show that it is possible to simulate equivalence class formation using the go/no-go procedure with compound stimuli and that it is necessary to conduct additional class training. The model developed is, therefore, an alternative to RELNET for the study of equivalence relations using computational simulations.

[article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4960284/pdf/40732_2016_Article_184.pdf)

In [1]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
import csv

from multiprocessing import Pool

%matplotlib inline
plt.style.use('seaborn')

# TensorFlow replica

[XOR example (simplest)](https://medium.com/@jaschaephraim/elementary-neural-networks-with-tensorflow-c2593ad3d60b)
[MNIST example](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb)

In [2]:
train_1_X=[
    [0.,0.,0.,1.,0.,1.,0.,0.,0.],# 0 0 0 1 0 1 0 0 0 
    [0.,1.,0.,0.,0.,1.,0.,0.,0.],# 0 1 0 0 0 1 0 0 0
    [0.,1.,0.,1.,0.,0.,0.,0.,0.],# 0 1 0 1 0 0 0 0 0
    [1.,0.,0.,0.,0.,0.,0.,1.,0.],# 1 0 0 0 0 0 0 1 0  
    [1.,0.,1.,0.,0.,0.,0.,0.,0.],# 1 0 1 0 0 0 0 0 0 
    [0.,0.,0.,0.,1.,0.,0.,1.,0.],# 0 0 0 0 1 0 0 1 0 
    [0.,0.,0.,0.,0.,0.,1.,1.,0.],# 0 0 0 0 0 0 1 1 0 
    [0.,0.,1.,0.,0.,0.,0.,0.,1.],# 0 0 1 0 0 0 0 0 1
    [0.,0.,0.,0.,0.,0.,0.,1.,1.],# 0 0 0 0 0 0 0 1 1 
    [0.,0.,1.,0.,0.,0.,1.,0.,0.],# 0 0 1 0 0 0 1 0 0
    [0.,0.,1.,0.,1.,0.,0.,0.,0.] # 0 0 1 0 1 0 0 0 0 
]

train_1_y=[1,1,1,1,0,1,0,1,0,1,0] 

test_1_X=[
    [1.,0.,0.,0.,1.,0.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.,1.,0.,0.],
    [0.,0.,0.,0.,0.,0.,1.,0.,1.],
    [0.,0.,0.,0.,1.,0.,0.,0.,1.]
]

test_1_y=[1,0,1,0]

train_2_X=[
    [1.,0.,0.,0.,1.,0.],# 1 0 0 0 1 0
    [1.,1.,0.,0.,0.,0.],# 1 1 0 0 0 0
    [0.,0.,1.,0.,1.,0.],# 0 0 1 0 1 0
    [0.,0.,0.,1.,1.,0.],# 0 0 0 1 1 0
    [0.,1.,0.,0.,0.,1.],# 0 1 0 0 0 1
    [0.,0.,0.,0.,1.,1.],# 0 0 0 0 1 1
    [0.,1.,0.,1.,0.,0.],# 0 1 0 1 0 0
    [0.,1.,1.,0.,0.,0.] # 0 1 1 0 0 0 
]

train_2_y=[1,0,1,0,1,0,1,0]

test_2_X=[
    [1.,0.,1.,0.,0.,0.],# 1 0 1 0 0 0
    [1.,0.,0.,1.,0.,0.],# 1 0 0 1 0 0 
    [0.,0.,0.,1.,0.,1.],# 0 0 0 1 0 1
    [0.,0.,1.,0.,0.,1.],# 0 0 1 0 0 1
]

test_2_y=[1,0,1,0]


In [3]:
def predict_net(train_in,test_in,train_out_list,hidden_units=4, report=False, max_epochs=20000):

    #variables
    input_units=len(train_in[0])
    train_out=[[float(i)]for i in train_out_list]#train_1_y
    output_units=1

    # tf session preparation
    x  = tf.placeholder("float", shape=[None, input_units])# Not in original code
    y_ = tf.placeholder("float", shape=[None, output_units])# Not in original code

    w1 = tf.Variable(tf.random_normal([input_units, hidden_units]))
    b1 = tf.Variable(tf.zeros([hidden_units]))

    w2 = tf.Variable(tf.random_normal([hidden_units, output_units]))
    b2 = tf.Variable(tf.zeros([output_units]))

    out1 = tf.sigmoid(tf.add(tf.matmul(x, w1), b1))# train_in
    out2 = tf.sigmoid(tf.add(tf.matmul(out1, w2), b2))

#     error = tf.subtract(y_, out2)
#     mse = tf.reduce_mean(tf.square(error))
    mse = tf.losses.mean_squared_error(y_, out2)

    train = tf.train.GradientDescentOptimizer(0.3).minimize(mse)

    # Trainning session

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    err, target = 1, 0.0025
    epoch = 0
    mserr=[]

    if report: 
        strt=time.time()

    while err > target and epoch < max_epochs:
        epoch += 1
        err, _ = sess.run([mse, train],feed_dict={x:train_in, y_:train_out})
        mserr.append(err)

    if report: 
        converg_time=time.time()-strt
        print("epoch:", epoch, "mse:", err, "time:", converg_time)

    train_prediction=out2.eval(feed_dict={x: train_in},session=sess)
    test_prediction=out2.eval(feed_dict={x: test_in},session=sess)
    sess.close()

    return mserr,train_prediction,test_prediction

## Individual Test

In [4]:
err_trn, pred_trn,pred_tst=predict_net(train_2_X,test_2_X,train_2_y,
                                           report=True, max_epochs=20000)

epoch: 5242 mse: 0.00249949 time: 9.175546169281006


In [5]:
pred_trn

array([[ 0.94807869],
       [ 0.06148046],
       [ 0.93710786],
       [ 0.0407777 ],
       [ 0.96815866],
       [ 0.0522774 ],
       [ 0.96185547],
       [ 0.05190805]], dtype=float32)

In [6]:
pred_tst

array([[ 0.08067271],
       [ 0.92915136],
       [ 0.1583064 ],
       [ 0.92925149]], dtype=float32)

## Iteraciones

In [ ]:
iteraciones=1000
#n_reports=20

n_iter_train_1=[]
predict_train_1=[]
predict_test_1=[]


n_iter_train_2=[]
predict_train_2=[]
predict_test_2=[]

converg_times=[]
filename="TF_replicas"


# with open(''.join(["Results_1/",filename,".csv"]), 'wb') as f:# 'wb' are the second parameter sugested values
#     writer = csv.writer(f)
#     writer.writerow(["exp_1_n_iter".encode(),
#                      "exp_1_pred_train_1".encode(),
#                      "exp_1_pred_train_2".encode(),
#                      "exp_1_pred_train_3".encode(),
#                      "exp_1_pred_train_4".encode(),
#                      "exp_1_pred_train_5".encode(),
#                      "exp_1_pred_train_6".encode(),
#                      "exp_1_pred_train_7".encode(),
#                      "exp_1_pred_train_8".encode(),
#                      "exp_1_pred_train_9".encode(),
#                      "exp_1_pred_train_10".encode(),
#                      "exp_1_pred_train_11".encode(),
#                      "exp_1_pred_test_1".encode(),
#                      "exp_1_pred_test_2".encode(),
#                      "exp_1_pred_test_3".encode(),
#                      "exp_1_pred_test_4".encode(),
#                      "exp_2_final_loss".encode(),
#                      "exp_2_n_iter".encode(),
#                      "exp_2_pred_train_1".encode(),
#                      "exp_2_pred_train_2".encode(),
#                      "exp_2_pred_train_3".encode(),
#                      "exp_2_pred_train_4".encode(),
#                      "exp_2_pred_train_5".encode(),
#                      "exp_2_pred_train_6".encode(),
#                      "exp_2_pred_train_7".encode(),
#                      "exp_2_pred_train_8".encode(),
#                      "exp_2_pred_test_1".encode(),
#                      "exp_2_pred_test_2".encode(),
#                      "exp_2_pred_test_3".encode(),
#                      "exp_2_pred_test_4".encode()
#                     ])

In [ ]:
for corrida in range(iteraciones):
    strt_time=time.time()
    
#     if __name__ == '__main__':
#         pool = Pool()
#         err_list_1, pred_train_1,pred_test_1=predict_net(train_1_X,test_1_X,train_1_y)
#         err_list_2, pred_train_2,pred_test_2=predict_net(train_2_X,test_2_X,train_2_y)    
#         pool.close() 
#         pool.join()
    
    err_list_1, pred_train_1,pred_test_1=predict_net(train_1_X,test_1_X,train_1_y)
    err_list_2, pred_train_2,pred_test_2=predict_net(train_2_X,test_2_X,train_2_y)    

    iter_vals=[item for sublist in [[len(err_list_1)],
                                    [float(p) for p in pred_train_1],
                                    [float(p) for p in pred_test_1],
                                    [err_list_2[-1]],
                                    [len(err_list_2)],
                                    [float(p) for p in pred_train_2],
                                    [float(p) for p in pred_test_2],
                                   ] for item in sublist]


#     converg_times.append(end_time-strt_time)

    with open(''.join(["Results_1/",filename,".csv"]), 'a') as f:# 'wb' are the second parameter sugested values
        writer = csv.writer(f)
        writer.writerow(iter_vals)
    
    end_time=time.time()
    print("iter: ", str(corrida), "| time: ", str(end_time-strt_time))


iter:  0 | time:  23.199451684951782
iter:  1 | time:  22.77290177345276
iter:  2 | time:  21.008889198303223
iter:  3 | time:  19.291786909103394
iter:  4 | time:  19.98091411590576
iter:  5 | time:  23.125616788864136
iter:  6 | time:  24.640780210494995
iter:  7 | time:  18.519185304641724
iter:  8 | time:  18.588013410568237
iter:  9 | time:  21.63394856452942
iter:  10 | time:  18.417426824569702
iter:  11 | time:  25.48310112953186
iter:  12 | time:  23.2302463054657
iter:  13 | time:  22.418742179870605
iter:  14 | time:  21.207468509674072
iter:  15 | time:  21.919699907302856
iter:  16 | time:  21.367149114608765
iter:  17 | time:  23.707412004470825
iter:  18 | time:  20.970123052597046
iter:  19 | time:  24.58641791343689
iter:  20 | time:  26.38665461540222
iter:  21 | time:  26.00441026687622
iter:  22 | time:  20.599992513656616
iter:  23 | time:  23.337913751602173
iter:  24 | time:  18.91368007659912
iter:  25 | time:  18.86335039138794
iter:  26 | time:  25.35812973976

# Results

In [ ]:
dat=pd.read_csv("Results_1/TF_replicas.csv", 
                header=None, 
                names=(["exp_1_n_epocs",
                        "XY_1",
                        "YZ_1",
                        "XZ_1",
                        "A1B1_1",
                        "A1B2_1",
                        "B1C1_1",
                        "B1C2_1",
                        "A2B2_1",
                        "A2B1_1",
                        "B2C2_1",
                        "B2C1_1",
                        "A1C1_1",
                        "A1C2_1",
                        "A2C2_1",
                        "A2C1_1",
                        "exp_2_final_loss",
                        "exp_2_n_epocs",
                        "A1B1_2",
                        "A1B2_2",
                        "B1C1_2",
                        "B1C2_2",
                        "A2B2_2",
                        "A2B1_2",
                        "B2C2_2",
                        "B2C1_2",
                        "A1C1_2",
                        "A1C2_2",
                        "A2C2_2",
                        "A2C1_2",
                       ]))# , sep=";" # para bases en español #_hid_4_1
#dat=pd.read_csv("Results_1/test_hid_4_1.csv")# , sep=";" # para bases en español #_hid_4_1
print(len(dat))
print(list(dat.columns))

In [ ]:
sns.distplot(dat["exp_1_n_epocs"])
#dat["exp_1_n_epocs"].plot.density(figsize=(12,8),alpha=0.6)
dat["exp_1_n_epocs"].describe()

In [ ]:
dat.plot.scatter(
    x='exp_2_n_epocs',
    y='exp_2_final_loss',
    ylim=(dat["exp_2_final_loss"].min(),dat["exp_2_final_loss"].max()),
    xlim=(dat["exp_2_n_epocs"].min(),dat["exp_2_n_epocs"].max()),
    figsize=(12,8),
    alpha=0.4
)
#dat['exp_1_loss'].plot.hist()

In [ ]:
dat["exp_2_final_loss"].describe()

In [ ]:
sns.distplot(dat["exp_2_n_epocs"])
#dat["exp_2_final_loss"].plot.density(figsize=(12,8),alpha=0.6)

In [ ]:
dat.iloc[:,1:12].describe()

In [ ]:
dat.iloc[:,1:12].plot(kind="box", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,1:12].plot(kind="density", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,12:16].describe()

In [ ]:
dat.iloc[:,12:16].plot(kind="box", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,12:16].plot.density(figsize=(16,8))

In [ ]:
dat.iloc[:,18:26].describe()

In [ ]:
dat.iloc[:,18:26].plot(kind="box", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,18:26].plot(kind="density", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,26:30].describe()

In [ ]:
dat.iloc[:,26:30].plot(kind="box", figsize=(16,8))#, alpha=0.6)

In [ ]:
dat.iloc[:,26:30].plot(kind="density", figsize=(16,8))#, alpha=0.6)